In [2]:
from __future__ import division
import itertools
from random import randint, seed
#from sage.all import gcd, inverse_mod

In [3]:
# https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm
def xgcd(b, n):
    x0, x1, y0, y1 = 1, 0, 0, 1
    while n != 0:
        q, b, n = b // n, n, b % n
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return  b, x0, y0

def gcd(a, b):
    return xgcd(a, b)[0]

def inverse_mod(r, q):
    assert gcd(r, q) == 1
    res = xgcd(r, q)[1]
    if res < 0:
        res += q
    assert res > 0 and res < q
    return res

In [9]:
def knapsack_brute_force(knapsack, x, solution):
    
    if x == 0:
        return [True, solution]
    
    if len(knapsack) == 0:
        if x == 0:
            return [True, solution]
        else:
            print 'c'
            return [False, []]
    
    if len(knapsack) == 1:
        if knapsack[0] == x:
            solution.append(x)
            return [True, solution]
        else:
            return [False, ]
    
    
    curr_elem = knapsack[0]
    
    if curr_elem > x:
        return knapsack_brute_force(knapsack[1:], x, solution)
    
    else:
        # try with taking the first element
        result = knapsack_brute_force(knapsack[1:], x - curr_elem, solution)
        if result[0]:
            solution.append(curr_elem)
            return [True, solution]
        else:
            # try with not taking the first element
            return knapsack_brute_force(knapsack[1:], x, solution)
        
def knapsack_brute_force_bits(knapsack, x):
    sol = knapsack_brute_force(knapsack, x, [])[1]
    res = []
    # dont ask my why in reversed order :D
    for elem in reversed(knapsack):
        if elem in sol:
            res.append(1)
        else:
            res.append(0)
    return res

In [65]:
wikipedia = 'Hands Across Hawthorne was a rally held at the Hawthorne Bridge in Portland, Oregon, on May 29, 2011, in response to an attack on a gay male couple one week earlier for holding hands while walking across the bridge. News of the attack spread throughout the Pacific Northwest and the United States. According to the couple and the Portland Police Bureau, a group of five men followed Brad Forkner and Christopher Rosevear along the bridge before they were physically assaulted. The assault was condemned by Portland s gay mayor, Sam Adams, and its police chief, Mike Reese. The attack prompted volunteers from the Q Center, an LGBT community center and non-profit organization, to form street patrols to monitor Portland s downtown area. Several LGBT and human rights organizations sponsored Hands Across Hawthorne in response to the attack, linking hands across the entire span of the Hawthorne Bridge to show solidarity. More than 4,000 people attended the rally.'
split2len(text_to_bytes(wikipedia, 100), 100)

['0000000001001000000000000110000100000000011011100000000001100100000000000111001100000000001000000000',
 '0000010000010000000001100011000000000111001000000000011011110000000001110011000000000111001100000000',
 '0010000000000000010010000000000001100001000000000111011100000000011101000000000001101000000000000110',
 '1111000000000111001000000000011011100000000001100101000000000010000000000000011101110000000001100001',
 '0000000001110011000000000010000000000000011000010000000000100000000000000111001000000000011000010000',
 '0000011011000000000001101100000000000111100100000000001000000000000001101000000000000110010100000000',
 '0110110000000000011001000000000000100000000000000110000100000000011101000000000000100000000000000111',
 '0100000000000110100000000000011001010000000000100000000000000100100000000000011000010000000001110111',
 '0000000001110100000000000110100000000000011011110000000001110010000000000110111000000000011001010000',
 '00000010000000000000010000100000000001110010000000000

In [116]:
def new_mochila(N=10, jump=10, sd=0):
    seed(sd)
    diff = [randint(1,jump) for k in range(N)]
    mochila = [diff[0]]
    for i in range(1,len(diff)):
        new_val = sum(mochila) + diff[i]
        mochila.append(new_val)
    return mochila

def split2len(s, n=16):
    def _f(s, n):
        while s:
            yield s[:n]
            s = s[n:]
    return list(_f(s, n))

def text_to_bytes(text, n=0):
    largo = len(text)
    if n > 0:
        text = text + ' '*(n - (largo % n))
    text = text.decode('utf8')
    text_e = ''.join(['{0:016b}'.format(ord(c)) for c in text])
    return text_e

def bytes_to_text(binary):
    return ''.join([unichr(int(ch,2)) for ch in split2len(binary)])

def select_q(mochila):
    suma = sum(mochila)
    return randint(suma+1,suma*10)

def select_r(q):
    try_r = randint(2,q-1)
    while gcd(try_r,q) != 1:
        try_r = randint(2,q)
    return try_r, inverse_mod(try_r,q)

def mochila_camuflada(mochila, q, r):
    return [(r*x)% q for x in mochila]

def codificar(mochila_camuflada, text):
    largo_mochila = len(mochila_camuflada)
    #assert largo_mochila % 16 == 0
    
    texto_codificado = text_to_bytes(text,largo_mochila)
    a_codifcar = split2len(texto_codificado, largo_mochila)
    return [sum([b for (a,b) in zip(k,mochila_camuflada) if a=='1']) for k in a_codifcar], texto_codificado


def unpack(n,l):
    i = len(l) -1
    for i in range(len(l)-1,-1,-1):
        if n>0:
            if l[i] <= n:
                n -= l[i]
                res = 1
            else:
                res = 0
            i-=1
        else:
            res = 0
        yield res

def decodificar(chunk, mochila_o, q, i_r):
    message = (chunk * i_r) % q
    int_message = list(unpack(message,mochila_o))[::-1]
    #print int_message
    #print ''.join(map(str,int_message))
    return bytes_to_text(''.join(map(str,int_message)))

def decodificar2(chunks, mochila_o, q, i_r):
    bits = []
    for chunk in chunks:
        message = (chunk * i_r) % q
        bits += list(unpack(message, mochila_o))[::-1]
        #print bits
    #print ''.join(map(str,bits))
    return bytes_to_text(''.join(map(str, bits)))

def decodificar_brute_force(chunk, mochi_c):
    int_message = list(knapsack_brute_force_bits(mochi_c, chunk))[::-1]
    return bytes_to_text(''.join(map(str,int_message)))
    


In [117]:
mochi = new_mochila(N = 10, jump = 100)
#print mochi
q = select_q(mochi)
r, i_r = select_r(q)
mochi_c = mochila_camuflada(mochi, q, r)
print mochi_c

#texto = 'Nó mé gústÁn lás tÍldÉs'
codifcado, x= codificar(mochi_c, wikipedia)
#print codifcado, x

[604753, 171500, 575404, 87558, 257159, 143072, 406053, 660642L, 365321L, 428817L]


In [118]:
decodificar2(codifcado, mochi, q, i_r)

u'Hands Across Hawthorne was a rally held at the Hawthorne Bridge in Portland, Oregon, on May 29, 2011, in response to an attack on a gay male couple one week earlier for holding hands while walking across the bridge. News of the attack spread throughout the Pacific Northwest and the United States. According to the couple and the Portland Police Bureau, a group of five men followed Brad Forkner and Christopher Rosevear along the bridge before they were physically assaulted. The assault was condemned by Portland s gay mayor, Sam Adams, and its police chief, Mike Reese. The attack prompted volunteers from the Q Center, an LGBT community center and non-profit organization, to form street patrols to monitor Portland s downtown area. Several LGBT and human rights organizations sponsored Hands Across Hawthorne in response to the attack, linking hands across the entire span of the Hawthorne Bridge to show solidarity. More than 4,000 people attended the rally.      '

In [114]:
s=''
for c in codifcado:
    s+=decodificar(c,mochi, q, i_r)
print s

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
0000000001
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
0010000000
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
0000011000
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0100000000
[0, 1, 1, 0, 1, 1, 1, 0, 0, 0]
0110111000
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
0000000110
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0100000000
[0, 0, 0, 1, 1, 1, 0, 0, 1, 1]
0001110011
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0000000000
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1000000000
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
0000010000
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0100000000
[0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
0110001100
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
0000000111
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
0010000000
[0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
0001101111
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
0000000001
[1, 1, 0, 0, 1, 1, 0, 0, 0, 0]
1100110000
[0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
0000011100
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
1100000000
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
0010000000
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
0000000100
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1000000000
[0, 0, 0, 1, 1, 0, 0, 0, 0, 1]
000

In [115]:
s=''
for c in codifcado:
    s+=decodificar_brute_force(c,mochi_c)
print s

ĀƸĀs ȀĀƌo̰̀ȀaͰ ƠπrˠĀǀa̰ Ƅ rȐ ưɀ ʀĀưĀ Ȑ Āhɐ Ġ@w̀ Ƽnɐ Ĉiɀ̀Ɣ iˠ ŀπr̀ Ƅ΀d ˀ ļeɰ̀Ƹ̀ ˰Ȁ̀aΐ Èɀ, ȀȀÀ@1 ˀ Ƥ΀ ̠Āǌ oˠ̀Ɣ t˰ Ƅ΀ Ȑ ǐ@cʰ Ƽ΀ Ȑ Ɯ@y ȀĀƄ̀e Ȁ̀ƼŀpˀĀπnɐ ǜŀeʰ Ɣ@rˀĀƔ ɠ̀ǈ h˰ Ɛɀnɰ Ơ@nɀ̀ǀhʐ Ɣ wȐ Ƭɀnɰ ƄÀr˰̀ǌ tʀĀrʐ Ɯŀ. ȀȀƔǀs Ȁ̀Ƙ tʀĀ@t̀Āƌˀ ̰ ǈŀaɀ ǐȀr˰ĀƜȀo͐ Āhɐ ŀ@cʐȀƤÀ à̀ǈĀhͰĀǌĀ ȐȀƐ tʀĀŀnʐ ƔĀ İ ƄĀe̰Ȁ@cȰ̀ǈĀiˠ̀Āo Ȁ Ơŀ Ȱ̀ǔ lɐ Ƅ΀d Ȁ Ơŀ Ā̀ǈĀlȐȀƐ P˰ ƤÀe ȀȀǔeȐĀ° a Ȁ̀ǈπu̀ Ƽƀ ɠĀǘŀ ːĀƸ f˰ ưπwɐ rȐ ƀo̠̀Ƹŀr ȀĀƸĀ 0 ǈɀs̀̀ǀȀe̠ ňπsɐȀƔ@r ȀĀưπnɰ ǐȀe ȀȀǈɀdɰĀeɠ̀ǈŀ ̀ Ɣɀ ͰĀǈŀ ̀ ǤÀiȰĀư̀y ȀĀǌÀa͐ ǐŀd ˠ ŐȀe ȀĀǌÀa͐ ǐ wȐ̀Àoˠ Ɣ̀nɐ y Ȁ ƼtˀĀƸĀ ̰ Ɯ@y ȀĀƄɀo̠ Àaː ĄĀaː̀° aˠ ɀt̰ ǀπlʐ̀Ɣ cʀĀƔƀ, ȀĀƤˀe ȀȀƔŀsɐȀĀhɐ ƄĀtȐ̀Ƭ p̠̀ƴ tɐ ƀoˀĀƸĀeɐȀǌ f̠̀ƴ tʀĀ@ 0ĀƸĀe̠ @n